LAYER 1 of Deepfake detection


---




Code for training the model for the first layer of classification that differentiates Real vs Fake images

The model of a Resnet 18 is used as the backbone. A classification head is an FCNN of 2 hidden layers of format NumFeatures(input)->256->64->1(sigmoid output)

Training details:

epochs = 150                 
batch size = 30                   
Loss function -> cross entropy               
optimiser -> SGD                 
learning rate -> 0.0001             
momentum = 0.99

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained ResNet-18 model
resnet18 = models.resnet18(pretrained=True)

# Freeze all layers
for param in resnet18.parameters():
    param.requires_grad = True

# Modify the last 10 layers to be trainable
# for param in resnet18.layer4.parameters():
#     param.requires_grad = True

# Modify the classification head
num_features = resnet18.fc.in_features
resnet18.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 2),  # Two output units
    nn.Softmax(dim=1)
)

# Move the model to the appropriate device (CPU or GPU)
resnet18 = resnet18.to(device)

# Specify the path to your parent folder containing "real" and "fake" subfolders
from google.colab import drive
drive.mount('/content/drive')
parent_folder = '/content/drive/MyDrive/diffusion_datasets/dalle'

# Define data transforms
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Load the dataset using ImageFolder (no need for manual label assignment)
dataset = datasets.ImageFolder(parent_folder, transform=transform)

# Split the dataset into training and validation sets
train_size = int(0.9*len(dataset))
# val_size = int(0.001*len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
batch_size = 30
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

# Specify loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet18.parameters(), lr=0.0001, momentum=0.99)

# Training loop
num_epochs = 150
for epoch in range(num_epochs):
    print("epoch start")
    resnet18.train()
    print("data loaded")
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = resnet18(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print("validation-->")
    # Validation loop (optional)
    resnet18.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = resnet18(images)
            val_loss = criterion(outputs, labels)
            break;
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}")

torch.save(resnet18.state_dict(), "Layer 1 model.pth")

Testing the accuracy of model trained on DALLE dataset - 95.2% accuracy

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torchvision.models import resnet18
from torch.utils.data import DataLoader

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the trained model
# model = torch.load("resnet18_model.pth")
model = resnet18(pretrained=True)  # Set pretrained to True if you used a pre-trained model
# model.fc = torch.nn.Linear(model.fc.in_features, 2)  # Adjust the output layer if necessary
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 2),  # Two output units
    nn.Softmax(dim=1)
)
# Move the model to the appropriate device (CPU or GPU)
model = model.to(device)

# Load the saved model state_dict
model_path = "Layer 1 model (1).pth"
djk = torch.load(model_path, map_location=torch.device('cpu'))
model.load_state_dict(djk)
model.eval()

# Specify the path to your test set
from google.colab import drive
drive.mount('/content/drive')
parent_folder = '/content/drive/MyDrive/diffusion_datasets/data'

# Define data transforms for the test set
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Load the test dataset using ImageFolder
test_dataset = datasets.ImageFolder(parent_folder, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

# Specify the binary cross-entropy loss function
criterion = nn.BCELoss()

# Initialize variables for accuracy calculation
correct_predictions = 0
total_samples = 0

# Evaluate accuracy on the test set
with torch.no_grad():
    count = 1
    for images, labels in test_loader:
        count += 1
        print(count)
        images, labels = images.to(device), labels.to(device)
        print(labels)
        outputs = model(images)
        # print(outputs)
        predictions = (outputs[:, 1] >= 0.5).float()  # Assuming index 1 corresponds to class 1
        print(predictions)
        correct_predictions += torch.sum(predictions == labels)
        total_samples += labels.size(0)

# Calculate accuracy
accuracy = correct_predictions / total_samples
print(f"Accuracy on the test set: {accuracy.item() * 100:.2f}%")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
2
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0], device='cuda:0')
tensor([1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
3
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Testing model trained on DALLE on biggan dataset - 83% accuracy

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torchvision.models import resnet18
from torch.utils.data import DataLoader

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the trained model
# model = torch.load("resnet18_model.pth")
model = resnet18(pretrained=True)  # Set pretrained to True if you used a pre-trained model
# model.fc = torch.nn.Linear(model.fc.in_features, 2)  # Adjust the output layer if necessary
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 2),  # Two output units
    nn.Softmax(dim=1)
)
# Move the model to the appropriate device (CPU or GPU)
model = model.to(device)

# Load the saved model state_dict
model_path = "Layer 1 model (1).pth"
djk = torch.load(model_path, map_location=torch.device('cpu'))
model.load_state_dict(djk)
model.eval()

# Specify the path to your test set
from google.colab import drive
drive.mount('/content/drive')
parent_folder = '/content/drive/MyDrive/CNN_synth_testset.zip (Unzipped Files)/biggan'

# Define data transforms for the test set
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Load the test dataset using ImageFolder
test_dataset = datasets.ImageFolder(parent_folder, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

# Specify the binary cross-entropy loss function
criterion = nn.BCELoss()

# Initialize variables for accuracy calculation
correct_predictions = 0
total_samples = 0

# Evaluate accuracy on the test set
with torch.no_grad():
    count = 1
    for images, labels in test_loader:
        count += 1
        print(count)
        images, labels = images.to(device), labels.to(device)
        print(labels)
        outputs = model(images)
        # print(outputs)
        predictions = (outputs[:, 1] >= 0.5).float()  # Assuming index 1 corresponds to class 1
        print(predictions)
        correct_predictions += torch.sum(predictions == labels)
        total_samples += labels.size(0)

# Calculate accuracy
accuracy = correct_predictions / total_samples
print(f"Accuracy on the test set: {accuracy.item() * 100:.2f}%")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
2
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0], device='cuda:0')
tensor([0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
3
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Layer 2 of Deepfake detection : differentiating between Gans and Diffusion models


---



Here below is the code to train the Layer 2 model

batch size = 30          
epoch = 120          
loss function -> Cross entropy loss      
optimizer -> SGD
learning rate = 0.0001             
momentum = 0.8

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained ResNet-18 model
resnet18 = models.resnet18(pretrained=True)

# Freeze all layers
for param in resnet18.parameters():
    param.requires_grad = True

# Modify the last 10 layers to be trainable
# for param in resnet18.layer4.parameters():
#     param.requires_grad = True

# Modify the classification head
num_features = resnet18.fc.in_features
resnet18.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 2),  # Two output units
    nn.Softmax(dim=1)
)

# Move the model to the appropriate device (CPU or GPU)
resnet18 = resnet18.to(device)

# Specify the path to your parent folder containing "real" and "fake" subfolders
from google.colab import drive
drive.mount('/content/drive')
parent_folder = '/content/drive/MyDrive/Deepfake'

# Define data transforms
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Load the dataset using ImageFolder (no need for manual label assignment)
dataset = datasets.ImageFolder(parent_folder, transform=transform)

# Split the dataset into training and validation sets
train_size = int(0.9*len(dataset))
# val_size = int(0.001*len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
batch_size = 30
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

# Specify loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet18.parameters(), lr=0.0001, momentum=0.8)

# Training loop
num_epochs = 120
for epoch in range(num_epochs):
    print("epoch start")
    resnet18.train()
    print("data loaded")
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        print(images)
        print(type(images))
        optimizer.zero_grad()
        outputs = resnet18(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print("validation-->")
    # Validation loop (optional)
    resnet18.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = resnet18(images)
            val_loss = criterion(outputs, labels)
            break;
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}")
    torch.save(resnet18.state_dict(), "resnet18_model.pth")

# Save the trained model
# torch.save(resnet18.state_dict(), "resnet18_model.pth")

Below is the code to test of Layer 2 model on a seperate dataset

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torchvision.models import resnet18
from torch.utils.data import DataLoader

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the trained model
model = resnet18(pretrained=False)  # Set pretrained to True if you used a pre-trained model
# model.fc = torch.nn.Linear(model.fc.in_features, 2)  # Change the output layer to have 2 units
num_features = model.fc.in_features

model.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 2),  # Two output units
    nn.Softmax(dim=1)
)

# # Move the model to the appropriate device (CPU or GPU)
model = model.to(device)

# Load the saved model state_dict
model_path = "Layer 2 model.pth"
model.load_state_dict(torch.load(model_path))
model.eval()

# Specify the path to your test set
from google.colab import drive
drive.mount('/content/drive')
parent_folder = '/content/drive/MyDrive/Deepfake test'

# Define data transforms for the test set
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Load the test dataset using ImageFolder
test_dataset = datasets.ImageFolder(parent_folder, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=80, shuffle=False, num_workers=4)

# Specify the binary cross-entropy loss function
criterion = nn.BCELoss()

# Initialize variables for accuracy calculation
correct_predictions = 0
total_samples = 0

# Evaluate accuracy on the test set
with torch.no_grad():
    count = 1
    for images, labels in test_loader:
        count += 1
        print(count)
        images, labels = images.to(device), labels.to(device)
        print(labels)
        outputs = model(images)
        # print(outputs)
        predictions = (outputs[:, 1] >= 0.5).float()  # Assuming index 1 corresponds to class 1
        print(predictions)
        correct_predictions += torch.sum(predictions == labels)
        total_samples += labels.size(0)

# Calculate accuracy
accuracy = correct_predictions / total_samples
print(f"Accuracy on the test set: {accuracy.item() * 100:.2f}%")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
2
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
3
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Below is the code to train model for Layer 3 : differentiating GANS

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained ResNet-18 model
resnet18 = models.resnet18(pretrained=True)

# Freeze all layers
for param in resnet18.parameters():
    param.requires_grad = True

# Modify the last 10 layers to be trainable
# for param in resnet18.layer4.parameters():
#     param.requires_grad = True

# Modify the classification head
num_features = resnet18.fc.in_features
resnet18.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 8),  # Two output units
    nn.Softmax(dim=1)
)

# Move the model to the appropriate device (CPU or GPU)
resnet18 = resnet18.to(device)

# Specify the path to your parent folder containing "real" and "fake" subfolders
from google.colab import drive
drive.mount('/content/drive')
parent_folder = '/content/drive/MyDrive/Layer3/made dataset for GANS'

# Define data transforms
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Load the dataset using ImageFolder (no need for manual label assignment)
dataset = datasets.ImageFolder(parent_folder, transform=transform)

# Split the dataset into training and validation sets
train_size = int(0.7*len(dataset))
# val_size = int(0.001*len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
batch_size = 30
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Specify loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet18.parameters(), lr=0.00075, momentum=0.9)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    print("epoch start")
    resnet18.train()
    print("data loaded")
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = resnet18(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print("validation-->")
    # Validation loop (optional)
    resnet18.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = resnet18(images)
            val_loss = criterion(outputs, labels)
            break;
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}")
    torch.save(resnet18.state_dict(), "Layer 3 model.pth")
# Save the trained model

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
epoch start
data loaded


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


validation-->
Epoch 1/100, Loss: 2.056751012802124, Val Loss: 2.020298719406128
epoch start
data loaded
validation-->
Epoch 2/100, Loss: 1.9796319007873535, Val Loss: 1.9476453065872192
epoch start
data loaded
validation-->
Epoch 3/100, Loss: 1.901429295539856, Val Loss: 1.8565815687179565
epoch start
data loaded
validation-->
Epoch 4/100, Loss: 1.6237565279006958, Val Loss: 1.7706815004348755
epoch start
data loaded
validation-->
Epoch 5/100, Loss: 1.572069764137268, Val Loss: 1.7198065519332886
epoch start
data loaded
validation-->
Epoch 6/100, Loss: 1.665627360343933, Val Loss: 1.6941735744476318
epoch start
data loaded
validation-->
Epoch 7/100, Loss: 1.5615119934082031, Val Loss: 1.6631779670715332
epoch start
data loaded
validation-->
Epoch 8/100, Loss: 1.733724594116211, Val Loss: 1.6447924375534058
epoch start
data loaded
validation-->
Epoch 9/100, Loss: 1.6112016439437866, Val Loss: 1.6088156700134277
epoch start
data loaded
validation-->
Epoch 10/100, Loss: 1.5651599168777466

below is the code to tarin model for layer 3 : differentiating Diffusion models


In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained ResNet-18 model
resnet18 = models.resnet18(pretrained=True)

# Freeze all layers
for param in resnet18.parameters():
    param.requires_grad = True

# Modify the last 10 layers to be trainable
# for param in resnet18.layer4.parameters():
#     param.requires_grad = True

# Modify the classification head
num_features = resnet18.fc.in_features
resnet18.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 8),  # Two output units
    nn.Softmax(dim=1)
)

# Move the model to the appropriate device (CPU or GPU)
resnet18 = resnet18.to(device)

# Specify the path to your parent folder containing "real" and "fake" subfolders
from google.colab import drive
drive.mount('/content/drive')
parent_folder = '/content/drive/MyDrive/Layer3/dataset for diffusion models'

# Define data transforms
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Load the dataset using ImageFolder (no need for manual label assignment)
dataset = datasets.ImageFolder(parent_folder, transform=transform)

# Split the dataset into training and validation sets
train_size = int(0.7*len(dataset))
# val_size = int(0.001*len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
batch_size = 30
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Specify loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet18.parameters(), lr=0.00075, momentum=0.9)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    print("epoch start")
    resnet18.train()
    print("data loaded")
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = resnet18(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print("validation-->")
    # Validation loop (optional)
    resnet18.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = resnet18(images)
            val_loss = criterion(outputs, labels)
            break;
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}")
    torch.save(resnet18.state_dict(), "Layer 3 model DM.pth")
# Save the trained model



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 59.0MB/s]


Mounted at /content/drive
epoch start
data loaded


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


validation-->
Epoch 1/100, Loss: 2.0110397338867188, Val Loss: 2.035123586654663
epoch start
data loaded
validation-->
Epoch 2/100, Loss: 1.9990819692611694, Val Loss: 2.002866506576538
epoch start
data loaded
validation-->
Epoch 3/100, Loss: 1.9190489053726196, Val Loss: 1.9808439016342163
epoch start
data loaded
validation-->
Epoch 4/100, Loss: 1.8708839416503906, Val Loss: 1.9628427028656006
epoch start
data loaded
validation-->
Epoch 5/100, Loss: 1.956441879272461, Val Loss: 1.948644995689392
epoch start
data loaded
validation-->
Epoch 6/100, Loss: 1.9958455562591553, Val Loss: 1.935645580291748
epoch start
data loaded
validation-->
Epoch 7/100, Loss: 1.917934775352478, Val Loss: 1.924179196357727
epoch start
data loaded
validation-->
Epoch 8/100, Loss: 1.8581454753875732, Val Loss: 1.9105859994888306
epoch start
data loaded
validation-->
Epoch 9/100, Loss: 1.8740558624267578, Val Loss: 1.8968579769134521
epoch start
data loaded
validation-->
Epoch 10/100, Loss: 1.7817634344100952,

below is the code to test the model for differentiating GANS

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torchvision.models import resnet18
from torch.utils.data import DataLoader

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the trained model
model = resnet18(pretrained=False)  # Set pretrained to True if you used a pre-trained model
# model.fc = torch.nn.Linear(model.fc.in_features, 8)  # Change the output layer to have 8 units
# model.fc = nn.Sequential(
#     model.fc,
#     nn.Softmax(dim=1)  # Add softmax activation
# )
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 8),  # Two output units
    nn.Softmax(dim=1)
)

# Move the model to the appropriate device (CPU or GPU)
model = model.to(device)

# Load the saved model state_dict
model_path = "Layer 3 model (1).pth"
model.load_state_dict(torch.load(model_path), strict=False)
model.eval()

# Specify the path to your test set
from google.colab import drive
drive.mount('/content/drive')
parent_folder = '/content/drive/MyDrive/Layer3/Test GANS subset'

# Define data transforms for the test set
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Load the test dataset using ImageFolder
test_dataset = datasets.ImageFolder(parent_folder, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=80, shuffle=False, num_workers=4)

# Initialize variables for accuracy calculation
correct_predictions = 0
total_samples = 0

# Evaluate accuracy on the test set
with torch.no_grad():
    count = 1
    for images, labels in test_loader:
        count += 1
        print(count)
        images, labels = images.to(device), labels.to(device)
        print(4-labels)
        outputs = model(images)
        # print(outputs)
        predictions = torch.argmax(outputs, dim=1)
        print(predictions)
        correct_predictions += torch.sum(predictions == (4-labels))
        total_samples += labels.size(0)

# Calculate accuracy
accuracy = correct_predictions / total_samples
print(f"Accuracy on the test set: {accuracy.item() * 100:.2f}%")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


2
tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3])
tensor([4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 0, 4,
        4, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3,
        7, 3, 3, 3, 3, 7, 3, 2])
3
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3])
tensor([3, 7, 3, 3, 7, 7, 3, 3, 3, 3, 3, 3, 3, 3, 3, 6, 3, 3, 3, 3, 7, 7, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 7, 7, 3, 3, 3, 6, 3, 3, 3,
        3, 7, 3, 

Below is the code to test the model for differentiating Diffusion models

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torchvision.models import resnet18
from torch.utils.data import DataLoader

# Check if GPU is available and set device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the trained model
model = resnet18(pretrained=False)  # Set pretrained to True if you used a pre-trained model
# model.fc = torch.nn.Linear(model.fc.in_features, 8)  # Change the output layer to have 8 units
# model.fc = nn.Sequential(
#     model.fc,
#     nn.Softmax(dim=1)  # Add softmax activation
# )
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 8),  # Two output units
    nn.Softmax(dim=1)
)

# Move the model to the appropriate device (CPU or GPU)
model = model.to(device)

# Load the saved model state_dict
model_path = "Layer 3 model DM.pth"
model.load_state_dict(torch.load(model_path), strict=False)
model.eval()

# Specify the path to your test set
from google.colab import drive
drive.mount('/content/drive')
parent_folder = '/content/drive/MyDrive/Layer3/test dataset for DM'

# Define data transforms for the test set
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Load the test dataset using ImageFolder
test_dataset = datasets.ImageFolder(parent_folder, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=80, shuffle=False, num_workers=4)

# Initialize variables for accuracy calculation
correct_predictions = 0
total_samples = 0

# Evaluate accuracy on the test set
with torch.no_grad():
    count = 1
    for images, labels in test_loader:
        count += 1
        print(count)
        images, labels = images.to(device), labels.to(device)
        print(labels)
        outputs = model(images)
        # print(outputs)
        predictions = torch.argmax(outputs, dim=1)
        print(predictions)
        correct_predictions += torch.sum(predictions == labels)
        total_samples += labels.size(0)

# Calculate accuracy
accuracy = correct_predictions / total_samples
print(f"Accuracy on the test set: {accuracy.item() * 100:.2f}%")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
2
tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([2, 1, 1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1,
        2, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 3, 1, 2, 1, 3, 1, 1, 1, 1,
        1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1,
        1, 3, 1, 1, 0, 2, 1, 1], device='cuda:0')
3
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0